In [139]:
from sympy import *
import itertools as it
import numpy as np

In [140]:
# Make symbols: methods to dynamically create as many as needed strings and Sympy symbols
# Shift is starting index; default 0

def make_strings(N,s, shift = 0):
	my_strings = []
	for i in range(shift, N + shift):
	    tmp_st = f'{s}{i}'
	    my_strings.append(tmp_st)
	return my_strings

def make_symbols(N,s, shift = 0):
	my_symbols = []
	for i in range(shift, N + shift):
	    tmp_st = f'{s}{i}'
	    globals()[tmp_st] = Symbol(tmp_st)
	    my_symbols.append(globals()[tmp_st])
	return my_symbols

# Generate harmonic game

Given measure, find harmonic game

# Parameters

In [141]:
# Game skeleton

#numPlayers = 4

#skeleton = np.random.randint(2, 6, numPlayers)

skeleton = [2, 2, 2]
skeleton

[2, 2, 2]

In [142]:
# N
numPlayers = len(skeleton)
players = range(numPlayers)

# -------------------------
# Number of action profiles
# -------------------------

# A
numPures = prod(skeleton)

# List of A_{-i} = for each player, number of action profiles of other players
numPuresMinus = [  int(numPures / skeleton[i]) for i in players ]

# -------------------------
# Number of payoff degrees of freedom
# -------------------------

# AN; number of payoff degrees of freedom
numPays = numPlayers * numPures

# sum_i A_i; number of harmonic measure dofs
numMeas = sum(skeleton)

In [143]:
# Pure actions: list of N lists, each with Ai elements; pure actions of each player

#pures_play = [ make_strings(skeleton[i], f'a{i}', shift = 1) for i in players ]
#pures_play = [ ['T','B'], ['L','R'], ['A','P'] ]
pures_play = [ ['1','2'], ['1','2'], ['1','2'] ]
#pures_play = [ ['T','B'], ['L','R'] ]

In [144]:
pures_play

[['1', '2'], ['1', '2'], ['1', '2']]

In [145]:
# Pure profiles; cartesian product of pure actions of each player
# Returns one list with A = numPures elements; each element is a tuple of strings

pures = list(it.product(*pures_play))

In [146]:
pures

[('1', '1', '1'),
 ('1', '1', '2'),
 ('1', '2', '1'),
 ('1', '2', '2'),
 ('2', '1', '1'),
 ('2', '1', '2'),
 ('2', '2', '1'),
 ('2', '2', '2')]

In [147]:
assert len(pures) == numPures

# Harmonic measure

In [148]:
# mu_values = [ np.random.randint(1, 5, skeleton[i]) for i in players]

#mu_values = [  [1, 0], [1, 2] ]

# mu_values = [  [2, 1], [3, 1], [4, 1] ]

# mu_values = [  [1, 2], [1, 3], [1,4], [1, 2, 3]  ]

# mu_values = [  [1, 1], [1, 1], [1, 1] ]

mu_values = [  [0, 1], [1, 2], [1, 3] ]

# mu_values = [ make_symbols(skeleton[i], f'mu{i}', shift = 1) for i in players ]


# SYMBOLIC
# mu_values = [ symbols('mu_T mu_B'), symbols('mu_L mu_R'), symbols('mu_A mu_P') ]

mu_values

[[0, 1], [1, 2], [1, 3]]

In [149]:
# Pack mu in list of dicts

mu = [   dict(zip( pures_play[i], mu_values[i] )) for i in players    ]
mu

[{'1': 0, '2': 1}, {'1': 1, '2': 2}, {'1': 1, '2': 3}]

In [150]:
numPures

8

In [151]:
numPuresMinus

[4, 4, 4]

In [152]:
numPays

24

In [153]:
numMeas

6

In [154]:
def print_payoff(payoff_dict, payoff_symbol):
    for i in players:
        for a in pures:
            print( f'{payoff_symbol}_{i}{a} = {payoff_dict[i][a]}' )
        print()

# Profiles of other players

In [155]:
# Pure profiles of other players
# Make list of N lists; the list pure_minus[i] contains the pure action profiles of players other than i
# Build taking the cartesian product of pure actions of all players other than i
# The size of the list pure_minus[i] is A_{-i} = \prod_{j \neq i} A_j

pures_minus = [ list( it.product( *( pures_play[:i] + pures_play[i+1:] ) ) ) for i in players ]

In [156]:
pures_minus

[[('1', '1'), ('1', '2'), ('2', '1'), ('2', '2')],
 [('1', '1'), ('1', '2'), ('2', '1'), ('2', '2')],
 [('1', '1'), ('1', '2'), ('2', '1'), ('2', '2')]]

In [157]:
numPuresMinus

[4, 4, 4]

In [158]:
for i in players:
    assert len(pures_minus[i]) == numPuresMinus[i]

# Make a

In [159]:
# Util to make (a_i, a_{-i}) given a_i and a_{-i} as tuple of strings (to be used as key for payoff dictionaries)
def make_pure(ai, a_minus_i, i):
    l = list(a_minus_i)
    return tuple(l[:i] + [ai] + l[i:])

In [160]:
def split_pure(a, i):
    
    ai = a[i]

    l = list(a)
    a_minus_i = tuple( l[:i] + l[i+1:] )
    return ai, a_minus_i

In [161]:
# check consistency
for i in players:
    for a in pures:
        ai, a_minus_i = split_pure(a, i)
        assert a == make_pure( ai, a_minus_i, i )

# Harmonic payoff

In [162]:
# harmonic payoff degrees of freedom; will build harmonic payoff h out of these

# to be determined harmonic payoff of player i (as many as A)

h_sym = [make_symbols(numPures, f'h{i}', shift = 1) for i in players]
h_sym

[[h01, h02, h03, h04, h05, h06, h07, h08],
 [h11, h12, h13, h14, h15, h16, h17, h18],
 [h21, h22, h23, h24, h25, h26, h27, h28]]

In [163]:
h = [  dict(zip(pures , h_sym[i] ))  for i in players]
h

[{('1', '1', '1'): h01,
  ('1', '1', '2'): h02,
  ('1', '2', '1'): h03,
  ('1', '2', '2'): h04,
  ('2', '1', '1'): h05,
  ('2', '1', '2'): h06,
  ('2', '2', '1'): h07,
  ('2', '2', '2'): h08},
 {('1', '1', '1'): h11,
  ('1', '1', '2'): h12,
  ('1', '2', '1'): h13,
  ('1', '2', '2'): h14,
  ('2', '1', '1'): h15,
  ('2', '1', '2'): h16,
  ('2', '2', '1'): h17,
  ('2', '2', '2'): h18},
 {('1', '1', '1'): h21,
  ('1', '1', '2'): h22,
  ('1', '2', '1'): h23,
  ('1', '2', '2'): h24,
  ('2', '1', '1'): h25,
  ('2', '1', '2'): h26,
  ('2', '2', '1'): h27,
  ('2', '2', '2'): h28}]

In [164]:
pures

[('1', '1', '1'),
 ('1', '1', '2'),
 ('1', '2', '1'),
 ('1', '2', '2'),
 ('2', '1', '1'),
 ('2', '1', '2'),
 ('2', '2', '1'),
 ('2', '2', '2')]

In [165]:
def is_unidev(a, b):
    if len(a) != len(b):
        return False

    differences = 0

    for ai, bi in zip(a, b):
        if ai != bi:
            differences += 1
            if differences > 1:
                return False

    return differences == 1

In [166]:
devs = [ ]
for i, a in enumerate(pures):
    for b in pures[i:]:
        if is_unidev(a,b):
            devs.append((a,b))

print(len(devs))
for d in devs:
    print(d)

12
(('1', '1', '1'), ('1', '1', '2'))
(('1', '1', '1'), ('1', '2', '1'))
(('1', '1', '1'), ('2', '1', '1'))
(('1', '1', '2'), ('1', '2', '2'))
(('1', '1', '2'), ('2', '1', '2'))
(('1', '2', '1'), ('1', '2', '2'))
(('1', '2', '1'), ('2', '2', '1'))
(('1', '2', '2'), ('2', '2', '2'))
(('2', '1', '1'), ('2', '1', '2'))
(('2', '1', '1'), ('2', '2', '1'))
(('2', '1', '2'), ('2', '2', '2'))
(('2', '2', '1'), ('2', '2', '2'))


In [167]:
def actor(d):
    assert d in devs
    a, b = d
    i = 0
    while a[i] == b[i]:
        i += 1
    return i

In [168]:
for d in devs:
    print(actor(d))

2
1
0
1
0
2
0
0
2
1
1
2


In [169]:
# deviazioni
dev_value = {}
for d in devs:
    a, b = d
    i = actor(d)
    dev_value[d] = h[i][a] - h[i][b]

In [170]:
dev_value

{(('1', '1', '1'), ('1', '1', '2')): h21 - h22,
 (('1', '1', '1'), ('1', '2', '1')): h11 - h13,
 (('1', '1', '1'), ('2', '1', '1')): h01 - h05,
 (('1', '1', '2'), ('1', '2', '2')): h12 - h14,
 (('1', '1', '2'), ('2', '1', '2')): h02 - h06,
 (('1', '2', '1'), ('1', '2', '2')): h23 - h24,
 (('1', '2', '1'), ('2', '2', '1')): h03 - h07,
 (('1', '2', '2'), ('2', '2', '2')): h04 - h08,
 (('2', '1', '1'), ('2', '1', '2')): h25 - h26,
 (('2', '1', '1'), ('2', '2', '1')): h15 - h17,
 (('2', '1', '2'), ('2', '2', '2')): h16 - h18,
 (('2', '2', '1'), ('2', '2', '2')): h27 - h28}

In [171]:
devs[0]

(('1', '1', '1'), ('1', '1', '2'))

In [172]:
dev_value[devs[0]]

h21 - h22

# System

In [173]:
pures

[('1', '1', '1'),
 ('1', '1', '2'),
 ('1', '2', '1'),
 ('1', '2', '2'),
 ('2', '1', '1'),
 ('2', '1', '2'),
 ('2', '2', '1'),
 ('2', '2', '2')]

In [174]:
pures_play

[['1', '2'], ['1', '2'], ['1', '2']]

In [175]:
pures_minus

[[('1', '1'), ('1', '2'), ('2', '1'), ('2', '2')],
 [('1', '1'), ('1', '2'), ('2', '1'), ('2', '2')],
 [('1', '1'), ('1', '2'), ('2', '1'), ('2', '2')]]

In [176]:
# --------------------------------
# KEY: HARMONIC FUNCTION
# --------------------------------
def Hi(a, i):
    ai, a_minus_i = split_pure(a, i)
    return sum(   [   mu[i][bi] * ( h[i][a] - h[i][ make_pure(bi, a_minus_i, i) ] )    for bi in pures_play[i]   ]  )

In [177]:
for i in players:
    for bi in pures_play[i]:
        print(mu[i][bi] + 1)

1
2
2
3
2
4


In [178]:
for i in players:
    for a in pures:
        print( Hi( a, i ) )

h01 - h05
h02 - h06
h03 - h07
h04 - h08
0
0
0
0
2*h11 - 2*h13
2*h12 - 2*h14
-h11 + h13
-h12 + h14
2*h15 - 2*h17
2*h16 - 2*h18
-h15 + h17
-h16 + h18
3*h21 - 3*h22
-h21 + h22
3*h23 - 3*h24
-h23 + h24
3*h25 - 3*h26
-h25 + h26
3*h27 - 3*h28
-h27 + h28


In [179]:
# --------------------------------
# KEY: HARMONIC SYSTEM in degrees of freedom h, that is payoff to be determined, prescribed the harmonic measure
# --------------------------------
system = [  sum( [  Hi(a,i)   for i in players ] )   for a in pures  ]

In [180]:
system

[h01 - h05 + 2*h11 - 2*h13 + 3*h21 - 3*h22,
 h02 - h06 + 2*h12 - 2*h14 - h21 + h22,
 h03 - h07 - h11 + h13 + 3*h23 - 3*h24,
 h04 - h08 - h12 + h14 - h23 + h24,
 2*h15 - 2*h17 + 3*h25 - 3*h26,
 2*h16 - 2*h18 - h25 + h26,
 -h15 + h17 + 3*h27 - 3*h28,
 -h16 + h18 - h27 + h28]

In [181]:
# one equation per pure in pures
assert len(system) == numPures

In [182]:
# Unknowns of system to solve: the weights (one per player) + the non-strategic dofs
dofs = list(it.chain(*h_sym))
dofs

[h01,
 h02,
 h03,
 h04,
 h05,
 h06,
 h07,
 h08,
 h11,
 h12,
 h13,
 h14,
 h15,
 h16,
 h17,
 h18,
 h21,
 h22,
 h23,
 h24,
 h25,
 h26,
 h27,
 h28]

In [183]:
numDofs = len(dofs)

In [184]:
# Unknowns of system to solve: harmonic weights
assert numDofs == numPays

In [185]:
# Matrix of linear system
A, b = linear_eq_to_matrix(system, *dofs)

In [186]:
A

Matrix([
[1, 0, 0, 0, -1,  0,  0,  0,  2,  0, -2,  0,  0,  0,  0,  0,  3, -3,  0,  0,  0,  0,  0,  0],
[0, 1, 0, 0,  0, -1,  0,  0,  0,  2,  0, -2,  0,  0,  0,  0, -1,  1,  0,  0,  0,  0,  0,  0],
[0, 0, 1, 0,  0,  0, -1,  0, -1,  0,  1,  0,  0,  0,  0,  0,  0,  0,  3, -3,  0,  0,  0,  0],
[0, 0, 0, 1,  0,  0,  0, -1,  0, -1,  0,  1,  0,  0,  0,  0,  0,  0, -1,  1,  0,  0,  0,  0],
[0, 0, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  2,  0, -2,  0,  0,  0,  0,  0,  3, -3,  0,  0],
[0, 0, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2,  0, -2,  0,  0,  0,  0, -1,  1,  0,  0],
[0, 0, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  1,  0,  0,  0,  0,  0,  0,  0,  3, -3],
[0, 0, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  1,  0,  0,  0,  0,  0,  0, -1,  1]])

In [187]:
sol = solve(system, dofs)

In [188]:
# Solution
sol

{h01: h05 - 2*h11 + 2*h13 - 3*h21 + 3*h22,
 h02: h06 - 2*h12 + 2*h14 + h21 - h22,
 h03: h07 + h11 - h13 - 3*h23 + 3*h24,
 h04: h08 + h12 - h14 + h23 - h24,
 h15: h17 + 3*h27 - 3*h28,
 h16: h18 - h27 + h28,
 h25: h26 - 2*h27 + 2*h28}

In [189]:
# Replace sol in dofs

extracted_sol = [e.subs(sol) for e in dofs]

In [190]:
extracted_sol

[h05 - 2*h11 + 2*h13 - 3*h21 + 3*h22,
 h06 - 2*h12 + 2*h14 + h21 - h22,
 h07 + h11 - h13 - 3*h23 + 3*h24,
 h08 + h12 - h14 + h23 - h24,
 h05,
 h06,
 h07,
 h08,
 h11,
 h12,
 h13,
 h14,
 h17 + 3*h27 - 3*h28,
 h18 - h27 + h28,
 h17,
 h18,
 h21,
 h22,
 h23,
 h24,
 h26 - 2*h27 + 2*h28,
 h26,
 h27,
 h28]

# General solution here

In [191]:
# zip solution in dictionary with dofs
sol_dict = dict(zip(dofs, extracted_sol))

In [192]:
sol_dict

{h01: h05 - 2*h11 + 2*h13 - 3*h21 + 3*h22,
 h02: h06 - 2*h12 + 2*h14 + h21 - h22,
 h03: h07 + h11 - h13 - 3*h23 + 3*h24,
 h04: h08 + h12 - h14 + h23 - h24,
 h05: h05,
 h06: h06,
 h07: h07,
 h08: h08,
 h11: h11,
 h12: h12,
 h13: h13,
 h14: h14,
 h15: h17 + 3*h27 - 3*h28,
 h16: h18 - h27 + h28,
 h17: h17,
 h18: h18,
 h21: h21,
 h22: h22,
 h23: h23,
 h24: h24,
 h25: h26 - 2*h27 + 2*h28,
 h26: h26,
 h27: h27,
 h28: h28}

In [193]:
dev_value

{(('1', '1', '1'), ('1', '1', '2')): h21 - h22,
 (('1', '1', '1'), ('1', '2', '1')): h11 - h13,
 (('1', '1', '1'), ('2', '1', '1')): h01 - h05,
 (('1', '1', '2'), ('1', '2', '2')): h12 - h14,
 (('1', '1', '2'), ('2', '1', '2')): h02 - h06,
 (('1', '2', '1'), ('1', '2', '2')): h23 - h24,
 (('1', '2', '1'), ('2', '2', '1')): h03 - h07,
 (('1', '2', '2'), ('2', '2', '2')): h04 - h08,
 (('2', '1', '1'), ('2', '1', '2')): h25 - h26,
 (('2', '1', '1'), ('2', '2', '1')): h15 - h17,
 (('2', '1', '2'), ('2', '2', '2')): h16 - h18,
 (('2', '2', '1'), ('2', '2', '2')): h27 - h28}

In [194]:
sol_dev_value = { d : dev_value[d].subs(sol_dict) for d in dev_value }

def print_dict(my_dict):
    for d in my_dict:
        print(d, ":", my_dict[d])
        print()
print_dict(sol_dev_value)

(('1', '1', '1'), ('1', '1', '2')) : h21 - h22

(('1', '1', '1'), ('1', '2', '1')) : h11 - h13

(('1', '1', '1'), ('2', '1', '1')) : -2*h11 + 2*h13 - 3*h21 + 3*h22

(('1', '1', '2'), ('1', '2', '2')) : h12 - h14

(('1', '1', '2'), ('2', '1', '2')) : -2*h12 + 2*h14 + h21 - h22

(('1', '2', '1'), ('1', '2', '2')) : h23 - h24

(('1', '2', '1'), ('2', '2', '1')) : h11 - h13 - 3*h23 + 3*h24

(('1', '2', '2'), ('2', '2', '2')) : h12 - h14 + h23 - h24

(('2', '1', '1'), ('2', '1', '2')) : -2*h27 + 2*h28

(('2', '1', '1'), ('2', '2', '1')) : 3*h27 - 3*h28

(('2', '1', '2'), ('2', '2', '2')) : -h27 + h28

(('2', '2', '1'), ('2', '2', '2')) : h27 - h28



In [195]:
devs

[(('1', '1', '1'), ('1', '1', '2')),
 (('1', '1', '1'), ('1', '2', '1')),
 (('1', '1', '1'), ('2', '1', '1')),
 (('1', '1', '2'), ('1', '2', '2')),
 (('1', '1', '2'), ('2', '1', '2')),
 (('1', '2', '1'), ('1', '2', '2')),
 (('1', '2', '1'), ('2', '2', '1')),
 (('1', '2', '2'), ('2', '2', '2')),
 (('2', '1', '1'), ('2', '1', '2')),
 (('2', '1', '1'), ('2', '2', '1')),
 (('2', '1', '2'), ('2', '2', '2')),
 (('2', '2', '1'), ('2', '2', '2'))]

In [196]:
# impose CLUB face
vertical_devs = [d for d in devs if actor(d) == 0]
vertical_devs

[(('1', '1', '1'), ('2', '1', '1')),
 (('1', '1', '2'), ('2', '1', '2')),
 (('1', '2', '1'), ('2', '2', '1')),
 (('1', '2', '2'), ('2', '2', '2'))]

In [197]:
vertical_devs_values = { d:sol_dev_value[d] for d in vertical_devs }
print_dict(vertical_devs_values)

(('1', '1', '1'), ('2', '1', '1')) : -2*h11 + 2*h13 - 3*h21 + 3*h22

(('1', '1', '2'), ('2', '1', '2')) : -2*h12 + 2*h14 + h21 - h22

(('1', '2', '1'), ('2', '2', '1')) : h11 - h13 - 3*h23 + 3*h24

(('1', '2', '2'), ('2', '2', '2')) : h12 - h14 + h23 - h24



In [198]:
simplify(sum(vertical_devs_values.values()))

-h11 - h12 + h13 + h14 - 2*h21 + 2*h22 - 2*h23 + 2*h24

In [199]:
mu

[{'1': 0, '2': 1}, {'1': 1, '2': 2}, {'1': 1, '2': 3}]

---

# Build solution instance

In [200]:
# fix remaining dofs; generate dictionary with dofs keys and random values
fix_remaining_dofs = dict(zip(dofs, np.random.randint(-5, 5, numDofs)))

In [201]:
fix_remaining_dofs

{h01: 3,
 h02: -1,
 h03: -5,
 h04: -4,
 h05: 2,
 h06: -2,
 h07: -5,
 h08: -2,
 h11: -1,
 h12: 0,
 h13: -1,
 h14: 2,
 h15: 4,
 h16: -3,
 h17: -4,
 h18: -1,
 h21: 3,
 h22: -3,
 h23: 0,
 h24: -4,
 h25: 3,
 h26: -5,
 h27: -2,
 h28: 1}

In [202]:
# fix remaining dofs to random number by subbing in sol_dict
sol_dict_instance = { key : sol_dict[key].subs(fix_remaining_dofs) for key in sol_dict}

In [203]:
sol_dict_instance

{h01: -16,
 h02: 8,
 h03: -17,
 h04: 0,
 h05: 2,
 h06: -2,
 h07: -5,
 h08: -2,
 h11: -1,
 h12: 0,
 h13: -1,
 h14: 2,
 h15: -13,
 h16: 2,
 h17: -4,
 h18: -1,
 h21: 3,
 h22: -3,
 h23: 0,
 h24: -4,
 h25: 1,
 h26: -5,
 h27: -2,
 h28: 1}

In [204]:
h

[{('1', '1', '1'): h01,
  ('1', '1', '2'): h02,
  ('1', '2', '1'): h03,
  ('1', '2', '2'): h04,
  ('2', '1', '1'): h05,
  ('2', '1', '2'): h06,
  ('2', '2', '1'): h07,
  ('2', '2', '2'): h08},
 {('1', '1', '1'): h11,
  ('1', '1', '2'): h12,
  ('1', '2', '1'): h13,
  ('1', '2', '2'): h14,
  ('2', '1', '1'): h15,
  ('2', '1', '2'): h16,
  ('2', '2', '1'): h17,
  ('2', '2', '2'): h18},
 {('1', '1', '1'): h21,
  ('1', '1', '2'): h22,
  ('1', '2', '1'): h23,
  ('1', '2', '2'): h24,
  ('2', '1', '1'): h25,
  ('2', '1', '2'): h26,
  ('2', '2', '1'): h27,
  ('2', '2', '2'): h28}]

In [205]:
for D in h:
    for key in D:
        print(key, ":", D[key].subs(sol_dict_instance))
    print()

('1', '1', '1') : -16
('1', '1', '2') : 8
('1', '2', '1') : -17
('1', '2', '2') : 0
('2', '1', '1') : 2
('2', '1', '2') : -2
('2', '2', '1') : -5
('2', '2', '2') : -2

('1', '1', '1') : -1
('1', '1', '2') : 0
('1', '2', '1') : -1
('1', '2', '2') : 2
('2', '1', '1') : -13
('2', '1', '2') : 2
('2', '2', '1') : -4
('2', '2', '2') : -1

('1', '1', '1') : 3
('1', '1', '2') : -3
('1', '2', '1') : 0
('1', '2', '2') : -4
('2', '1', '1') : 1
('2', '1', '2') : -5
('2', '2', '1') : -2
('2', '2', '2') : 1



# Result: harmonic game

In [206]:
print(skeleton)

[2, 2, 2]


In [207]:
sol_dict_instance.values()

dict_values([-16, 8, -17, 0, 2, -2, -5, -2, -1, 0, -1, 2, -13, 2, -4, -1, 3, -3, 0, -4, 1, -5, -2, 1])

# End